In [ ]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from scipy import stats

# Download historical price data for Gas, Ruble, and USDT
start_date = '2018-01-01'
end_date = '2025-07-27'
gas_ticker = 'NG=F'  # NYMEX Natural Gas Futures
ruble_ticker = 'RUB=X'  # USD/RUB exchange rate
usdt_ticker = 'USDT-USD'  # Tether USD
gas = yf.download(gas_ticker, start=start_date, end=end_date, interval='1d')[['Close']]
ruble = yf.download(ruble_ticker, start=start_date, end=end_date, interval='1d')[['Close']]
usdt = yf.download(usdt_ticker, start=start_date, end=end_date, interval='1d')[['Close']]

# Rename columns for clarity
gas.columns = ['Gas']
ruble.columns = ['Ruble']
usdt.columns = ['USDT']

# Merge dataframes on date
data = gas.join(ruble, how='inner').join(usdt, how='inner')
data = data.dropna()

# First stage: regress Ruble on Gas (IV)
X = data['Ruble']
IV = data['Gas']
IV_const = sm.add_constant(IV)
first_stage = sm.OLS(X, IV_const).fit()
residuals = first_stage.resid

# Second stage: regress USDT on Ruble and residuals
X_ols = pd.DataFrame({'Ruble': X, 'Ruble_resid': residuals})
X_ols_const = sm.add_constant(X_ols)
y = data['USDT']
ols_model = sm.OLS(y, X_ols_const).fit()
print(ols_model.summary())
print('Coefficient for Ruble residuals:', ols_model.params['Ruble_resid'])